In [1]:
!pip install transformers

In [2]:
!pip install torch

In [1]:
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder
import pandas as pd

In [2]:
# 1. 데이터 불러오기
data = pd.read_csv('spam.csv', encoding='latin-1')
data = data[['v1', 'v2']]
data.columns = ['label', 'message']

In [3]:
# 2. 레이블 인코딩
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['label'])

In [19]:
# 데이터 샘플링 (전체 데이터의 10% 사용)
data_sample = data.sample(frac=0.2, random_state=42)

In [20]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(data_sample['message'], data_sample['label'], test_size=0.2, random_state=42)

In [21]:
# BERT 모델과 토크나이저 로드
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
# 데이터 토크나이징
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, return_tensors='pt')
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True, return_tensors='pt')

class SpamDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SpamDataset(train_encodings, y_train.tolist())
test_dataset = SpamDataset(test_encodings, y_test.tolist())

In [23]:
# 훈련 인자 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir='./logs',
)

In [24]:
# Trainer 객체 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [25]:
# 모델 훈련
trainer.train()

Step,Training Loss


TrainOutput(global_step=336, training_loss=0.05806075391315278, metrics={'train_runtime': 3082.9906, 'train_samples_per_second': 0.867, 'train_steps_per_second': 0.109, 'total_flos': 303571060675740.0, 'train_loss': 0.05806075391315278, 'epoch': 3.0})

In [28]:
# 예측하기
def classify_email(email):
    encoding = tokenizer(email, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        output = model(**encoding)
    prediction = output.logits.argmax(-1).item()
    return label_encoder.inverse_transform([prediction])[0]

In [29]:
# 예시 이메일
new_email = "Congratulations! You've won a $1,000 Walmart gift card."
print("분류 결과:", classify_email(new_email))

분류 결과: ham
